# 2-keywords-matching (archived)
- Note: this notebooks implemented a naive way of matching keywords for both pdfs and website. Since we are not using direct keywords matching for now, this notebook is archived.

In [227]:
import PyPDF2
from tqdm import tqdm
import re
import numpy as np
import pandas as pd
import os

In [2]:
keywords = ["minimum lot size",
            "minimum lot area",
            "dwelling", 
            "residential", 
            "residence", 
            "residential district",
            "floor to area ratio",
            "floor-to-area ratio",
            #"maximum lot coverage",
            "side setback",
            "maximum density",
            "duplex",
            "rowhouse",
            #"condominium",
            "multifamily",
            "multi-family",
            "single family",
            "single-family",
            "townhouse",
            "primary use",
            "residential type",
            "minimum parking per unit",
            "minimum unit size",
            "public hearing"
        ]

In [9]:
len(keywords)

22

In [3]:
def calc_relevance_scores(text, return_keywords = False):
    found_count = 0
    keyword_lst = []
    for k in keywords:
        fc = len(re.findall(k, text))
        if fc > 0:
            found_count += fc
            keyword_lst.append(k)
    if return_keywords:
        return found_count, keyword_lst
    return found_count

In [4]:
def process_pdf_page(text):
    text = text.lower()
    text = text.replace('\n-\n', '-')
    return text

def pdf_2_text(filepath, k = 1):
    pdf_file = open(filepath, 'rb')
    pdfReader = PyPDF2.PdfFileReader(pdf_file)
    pages = []
    for page_idx in tqdm(range(pdfReader.numPages)):
        pageObj = pdfReader.getPage(page_idx)
        text = pageObj.extractText()
        text = process_pdf_page(text)
        pages.append(text)
    if k == 1:
        return pages
    agg_pages = []
    for p_idx in range(len(pages) - k):
        agg_pages.append('\n\n'.join(pages[p_idx: p_idx + k - 1]))
    return agg_pages

def calc_runing_sum(lst, k = 5):
    curr_sum = sum(lst[:k])
    r_sum = [curr_sum]
    for i in range(1, len(lst) - k):
        curr_sum -= lst[i - 1]
        curr_sum += lst[i + k - 1]
        r_sum.append(curr_sum)
    return r_sum

In [5]:
file_lst = ['Anne Arundel.pdf', 'Carroll.pdf', 'Calvert.pdf', 'Rockville.pdf', 'Cecil.pdf']
page_dict = {}
for fp in file_lst:
    pages = pdf_2_text('../data/md/' + fp, k = 3)
    page_dict[fp] = pages

100%|███████████████████████████████| 468/468 [00:17<00:00, 26.42it/s]


In [6]:
results = {}
for fp in page_dict.keys():
    pages = page_dict[fp]
    found_counts_dict = {}
    for i in range(len(pages)):
        found_counts_dict[i] = calc_relevance_scores(pages[i], True)
    results[fp] = sorted(found_counts_dict.items(), key = lambda x: x[1][0], reverse = True)

In [12]:
results['Carroll.pdf'][:20]
[(i[0], i[1][0]) for i in results['Carroll.pdf'][:20]]

[(71, 33),
 (55, 32),
 (67, 30),
 (54, 29),
 (75, 29),
 (77, 29),
 (78, 26),
 (43, 25),
 (63, 25),
 (62, 24),
 (8, 23),
 (74, 23),
 (66, 22),
 (72, 19),
 (76, 19),
 (42, 18),
 (70, 18),
 (7, 16),
 (64, 14),
 (68, 14)]

In [8]:
results['Rockville.pdf'][:20]

[(124, (46, ['minimum lot area', 'dwelling', 'residential', 'duplex'])),
 (123, (31, ['minimum lot area', 'dwelling', 'residential', 'residence'])),
 (133, (30, ['dwelling', 'residential', 'duplex', 'townhouse'])),
 (125, (26, ['dwelling', 'residential', 'duplex'])),
 (132, (25, ['dwelling', 'residential', 'townhouse'])),
 (127, (23, ['dwelling', 'residential'])),
 (27, (21, ['dwelling', 'duplex', 'townhouse'])),
 (137, (21, ['dwelling', 'residential', 'townhouse'])),
 (204, (21, ['minimum lot size', 'dwelling', 'residential', 'townhouse'])),
 (126, (20, ['dwelling', 'residential'])),
 (186, (20, ['residential', 'townhouse'])),
 (136, (19, ['dwelling', 'residential', 'townhouse'])),
 (187, (19, ['dwelling', 'residential', 'townhouse'])),
 (188, (19, ['dwelling', 'residential', 'townhouse'])),
 (201, (19, ['residential'])),
 (225, (19, ['dwelling', 'residential', 'townhouse'])),
 (226, (18, ['dwelling', 'residential', 'townhouse'])),
 (131, (17, ['dwelling', 'residential'])),
 (134, (17

In [11]:
[(i[0], i[1][0]) for i in results['Rockville.pdf'][:20]]

[(124, 46),
 (123, 31),
 (133, 30),
 (125, 26),
 (132, 25),
 (127, 23),
 (27, 21),
 (137, 21),
 (204, 21),
 (126, 20),
 (186, 20),
 (136, 19),
 (187, 19),
 (188, 19),
 (201, 19),
 (225, 19),
 (226, 18),
 (131, 17),
 (134, 17),
 (202, 17)]

'abcdefghijklmnopqrstuvwxyz'

# matching scrapped txt files

In [216]:
from nltk.tokenize import word_tokenize

In [217]:
keywords = open('../data/keywords.txt', 'r').readlines()
keywords = [w.replace('\n', '').lower() for w in keywords]
sub_keywords = {k[:-4]: int(k[-2]) for k in keywords[5:]}
keywords = keywords[:5]

sub_keywords_re_pattern = '|'.join(['(' + k + ')' for k in sub_keywords])
sub_keywords, keywords

({'minimum': 3,
  'lot size': 2,
  'lot area': 2,
  'building area': 2,
  'acreage': 2,
  'acre': 2,
  'square feet': 2,
  'shall be': 1,
  'requirement': 1,
  'permitted': 1,
  'square feet per dwelling unit': 3},
 ['lot area',
  'lot size',
  'building area ',
  'building site area acres/dwelling unit',
  'square feet/dwelling unit'])

In [218]:
tmp = pd.read_csv('../data/tmp.csv')
nums = [0.5, 0.25, 0.75]
alphabets = ['half', 'quarter', 'three-quarters']
for i in range(1, 6):
    numbers = tmp['num' + str(i)].values
    words = tmp['alpha' + str(i)].values
    nums += list(numbers)
    alphabets += list(words)
alphabets = [a.lower() for a in alphabets]
alphabets_2_numeric = dict(zip(alphabets, nums))
alphabets_keys = set(alphabets_2_numeric.keys())
# alphabets_keys
# alphabets_2_numeric

In [219]:
start_page_token = '[[START-PAGE]]'
start_section_token = '[[SECTIONTITLE]]'
start_content_token = '[[CONTENT]]'
special_tokens = [start_page_token, start_section_token, start_content_token]

In [223]:
def replace_numeric_tokens(doc):
    tokens = word_tokenize(doc)
    for i in range(len(tokens)):
        if tokens[i].strip().lower() in alphabets_keys:
            tokens[i] = str(alphabets_2_numeric[tokens[i].strip().lower()])
    return ' '.join(tokens)

In [232]:
for filepath in os.listdir('../data/scrapped'):
    sample_file = '../data/scrapped/' + filepath
    f = open(sample_file, 'r')
    lines = f.readlines()
    f.close()
    
    ########## sentence locations ##########
    found_sents_idx = []
    for i in tqdm(range(len(lines))):
        l = lines[i].lower().strip()
        for w in keywords:
            if w in l:
                tmp_i = i - 1
                start = tmp_i
                while tmp_i >= 0:
                    if lines[tmp_i].strip() in special_tokens:
                        start = tmp_i + 1
                        break
                    tmp_i -= 1
                tmp_i = i + 1
                end = tmp_i
                while tmp_i < len(lines):
                    if lines[tmp_i].strip() in special_tokens:
                        end = tmp_i
                        break
                    tmp_i += 1 
                if (start, end, w) not in found_sents_idx:
                    found_sents_idx.append((start, end, w))
    # found_sents_idx = np.unique(found_sents_idx)
    print('found_sents_idx:', len(found_sents_idx))
    
    ########## keyword locations (token level) ##########
    keyword_locs = []
    for idx_pair in found_sents_idx:
        start, end, w = idx_pair
        doc = replace_numeric_tokens(' '.join(lines[start: end+1]))
        keywords_loc_dict = {}
        try:
            match_spans = [m.span() for m in re.finditer(w, doc)]
        except:
            print('ERROR, not found!')
            continue
        keywords_loc_dict[w] = match_spans

        for span in match_spans:
            span_start, span_end = span
            start = max([span_start - 200, 0])
            end = min([span_end + 200, len(doc)])
            tmp_doc = doc[start: end + 1]
            tokens = word_tokenize(tmp_doc)
            numeric_values = []
            numeric_idx = -1
            for t in tokens:
                numeric_idx += 1
                try:
                    numeric_values.append((float(t), numeric_idx))
                except:
                    continue

            new_span_start = span_start
            if start != 0:
                new_span_start = 200
            span_token_idx = len(word_tokenize(tmp_doc[:new_span_start]))
            if len(numeric_values) > 0:
                keyword_locs.append([w, tmp_doc, numeric_values, span_token_idx])
    
    
    ########## final results ##########
    threshold = 12
    selected_docs = []
    for k_loc in keyword_locs:
        w, doc, loc_lst, span_idx = k_loc
        sub_keywords_match = list(re.finditer(sub_keywords_re_pattern, doc))
        if len(sub_keywords_match) > 0:
            match_score = sum([sub_keywords[m.group()] for m in sub_keywords_match])
            if match_score >= threshold:
                selected_docs.append((w, doc, loc_lst, span_idx, match_score))

    final_results = []
    for doc in selected_docs:
        w, doc, loc_lst, span_idx, match_score = doc
        best_num = []
        if span_idx < loc_lst[0][1]:
            best_num.append(loc_lst[0][0])
        elif span_idx > loc_lst[-1][1]:
            best_num.append(loc_lst[-1][0])
        else:
            for i in range(len(loc_lst) - 1):
                if loc_lst[i][1] <= span_idx and loc_lst[i+1][1] >= span_idx:
                    best_num.append(loc_lst[i][0])
                    best_num.append(loc_lst[i+1][0])
        final_results.append({
            'keyword': w,
            'best_num': best_num,
            'doc': doc,
            'match_score': match_score
        })
    if len(final_results) == 0:
        print('Error: no result for', filepath)
        continue
    final_results_df = pd.DataFrame(final_results).sort_values(by = ['keyword', 'match_score'], ascending = False)
    final_results_df.to_csv('../data/results/' + filepath.replace('txt', 'csv'), index = False)

100%|███████████████████████| 23998/23998 [00:00<00:00, 439354.98it/s]


found_sents_idx: 51


100%|███████████████████████| 20016/20016 [00:00<00:00, 734620.71it/s]


found_sents_idx: 27


100%|███████████████████████| 25463/25463 [00:00<00:00, 664102.05it/s]


...

found_sents_idx: 84


100%|███████████████████████| 26927/26927 [00:00<00:00, 790315.41it/s]


found_sents_idx: 53


100%|███████████████████████| 29556/29556 [00:00<00:00, 783355.87it/s]

found_sents_idx: 1
Error: no result for neptune.txt


In [225]:
# found_sents_idx = []

# for i in tqdm(range(len(lines))):
#     l = lines[i].lower().strip()
#     for w in keywords:
#         if w in l:
#             tmp_i = i - 1
#             start = tmp_i
#             while tmp_i >= 0:
#                 if lines[tmp_i].strip() in special_tokens:
#                     start = tmp_i + 1
#                     break
#                 tmp_i -= 1
#             tmp_i = i + 1
#             end = tmp_i
#             while tmp_i < len(lines):
#                 if lines[tmp_i].strip() in special_tokens:
#                     end = tmp_i
#                     break
#                 tmp_i += 1 
#             if (start, end, w) not in found_sents_idx:
#                 found_sents_idx.append((start, end, w))
# # found_sents_idx = np.unique(found_sents_idx)
# len(found_sents_idx)

In [221]:
# found_sents_idx

In [226]:
# s = '23.5'
# str.isalnum(s)
# # int(s)

In [187]:
# keyword_locs = []

# for idx_pair in found_sents_idx:
#     start, end, w = idx_pair
#     doc = replace_numeric_tokens(' '.join(lines[start: end+1]))
#     keywords_loc_dict = {}
#     try:
#         match_spans = [m.span() for m in re.finditer(w, doc)]
#     except:
#         print('ERROR, not found!')
#         continue
#     keywords_loc_dict[w] = match_spans
    
#     for span in match_spans:
#         span_start, span_end = span
#         start = max([span_start - 200, 0])
#         end = min([span_end + 200, len(doc)])
#         tmp_doc = doc[start: end + 1]
#         tokens = word_tokenize(tmp_doc)
#         numeric_values = []
#         numeric_idx = -1
#         for t in tokens:
#             numeric_idx += 1
#             try:
#                 numeric_values.append((float(t), numeric_idx))
#             except:
#                 continue
        
#         new_span_start = span_start
#         if start != 0:
#             new_span_start = 200
#         span_token_idx = len(word_tokenize(tmp_doc[:new_span_start]))
#         if len(numeric_values) > 0:
#             keyword_locs.append([w, tmp_doc, numeric_values, span_token_idx])

In [188]:
len(keyword_locs)
keyword_locs[0]

['lot area',
 'bandoned or ceased for a period of 12 consecutivemonths , it shall be considered abandoned and shall terminate . ( 4 ) To authorize , upon appeal , in specific cases suchvariances from the setback or lot area provisions of this Title aswill not be contrary to the public interest where , owing to specialor unique conditions , a literal enforcement of the provisions of this Title would result in unnecessary ',
 [(12.0, 7), (4.0, 20)],
 36]

In [228]:
# threshold = 12
# selected_docs = []
# for k_loc in keyword_locs:
#     w, doc, loc_lst, span_idx = k_loc
#     sub_keywords_match = list(re.finditer(sub_keywords_re_pattern, doc))
#     if len(sub_keywords_match) > 0:
#         match_score = sum([sub_keywords[m.group()] for m in sub_keywords_match])
#         if match_score >= threshold:
#             selected_docs.append((w, doc, loc_lst, span_idx, match_score))

# final_results = []
# for doc in selected_docs:
#     w, doc, loc_lst, span_idx, match_score = doc
#     best_num = []
#     if span_idx < loc_lst[0][1]:
#         best_num.append(loc_lst[0][0])
#     elif span_idx > loc_lst[-1][1]:
#         best_num.append(loc_lst[-1][0])
#     else:
#         for i in range(len(loc_lst) - 1):
#             if loc_lst[i][1] <= span_idx and loc_lst[i+1][1] >= span_idx:
#                 best_num.append(loc_lst[i][0])
#                 best_num.append(loc_lst[i+1][0])
#     final_results.append({
#         'keyword': w,
#         'best_num': best_num,
#         'doc': doc,
#         'match_score': match_score
#     })
# final_results_df = pd.DataFrame(final_results).sort_values(by = ['keyword', 'match_score'], ascending = False)

In [214]:
# final_results_df = final_results_df.groupby('keyword').agg('max')
final_results_df.to_csv('../data/worcester_result.csv', index = False)

In [215]:
final_results_df.groupby('keyword').agg('max')

,best_num,doc,match_score
keyword,,,
building area,"[300.0, 2.0]",feet of an amenity area and 1 per each 5 resid...,20
lot area,[50.0],"unit . ( 1 ) In the E-1 District , 1 unit per ...",18


In [ ]:
meta_data = pd.read_csv('../data/ordinates_meta_data_sub.csv')
meta_data.head()

In [ ]:
for row_idx in range(meta_data.shape[0]):
    curr_row = meta_data.iloc[row_idx]
    print('=> Processing:', curr_row.county)
    f = open('../data/scrapped/' + curr_row.county.replace(' ', '_').lower() + '.txt', 'r')
    
    f.close()
    print(str(ts.root_node)[:3000])